# Query Optimization

## Pre-Requisites
1. Load the Account Dataset from `initialize_database.ipynb`

In [ ]:
### SELECT 
from LabDatabaseHandler import LabDatabaseHandler
db = LabDatabaseHandler()
sql = 'select * from account_dataset'

db.execute(sql)